# Dense Array Basics

In this tutorial you will learn how to:
* create a dense array
* inspect the array schema
* write to and read from the array
* write and read array metadata
* create arrays with multiple attributes and var-sized attributes
* treat dense arrays as dataframes and even run SQL queries

## Necessary Libraries

You need to install [TileDB-Py](https://github.com/TileDB-Inc/TileDB-Py), the Python wrapper of [TileDB Embedded](https://github.com/TileDB-Inc/TileDB), as follows:

```bash
# Pip:
$ pip install tiledb

# Or Conda:
$ conda install -c conda-forge tiledb-py
```

You'll also need to install pandas and numpy

```bash
# Pip:
$ pip install numpy pandas

# Or Conda:
$ conda install numpy pandas
```

Note that the TileDB core is a C++ library. To boost performance when integrating with pandas, we use Apache Arrow to achieve zero-copy when returning results from TileDB into pandas dataframes. You need to **install pyarrow** to take advantage of this optimization.

```bash
# Pip:
$ pip install pyarrow

# Or Conda:
$ conda install -c conda-forge pyarrow
```

One of the cool things about TileDB is that it offers a powerful integration with embedded MariaDB. This allows for execution of arbitrary SQL queries directly on TileDB arrays (both dense and sparse). We took appropriate care to push the fast slicing, column subselecting and column conditions of the query down to TileDB, leaving the rest of the SQL execution to MariaDB.

To install this capability, run:
```bash
conda install -c conda-forge libtiledb-sql-py
```

## Setup

We first start by importing the libraries we will use in this tutorial (ignore any thrown mysql errors/warnings, they are benign).

In [1]:
import pandas as pd, numpy as np
import shutil, urllib.request, os.path
import tiledb, tiledb.sql

print("TileDB core version: {}".format(tiledb.libtiledb.version()))
print("TileDB-Py version: {}".format(tiledb.version()))
db = tiledb.sql.connect()
print("TileDB-SQL-Py version: {}".format(pd.read_sql("SELECT PLUGIN_AUTH_VERSION FROM information_schema.PLUGINS WHERE PLUGIN_NAME='mytile'", con=db)['PLUGIN_AUTH_VERSION'][0]))

TileDB core version: (2, 11, 1)
TileDB-Py version: (0, 17, 2)
TileDB-SQL-Py version: 0.18.1


Got ERROR: "Could not open mysql.plugin table: "Table 'mysql.plugin' doesn't exist". Some plugins may be not loaded" errno: 2000
Got ERROR: "Can't open and lock privilege tables: Table 'mysql.servers' doesn't exist" errno: 2000
Got ERROR: "Can't open the mysql.func table. Please run mysql_upgrade to create it." errno: 2000
/home/jdblischak/miniconda3/envs/tiledb/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Before we start, we create the TileDB context passing a **configuration parameter** around memory allocation during read queries that will be explained in a later tutorial. That needs to be set at the *very beginning* of the code and before any other TileDB function is called.

In [2]:
# Sets the buffer size parameter to 50MB. TileDB will allocate 50MB per attribute at the
# start of the query, but if the result is larger, TileDB will automatically expand 
# the buffers to fit the whole result.
cfg = tiledb.Ctx().config()
cfg.update(
  {
    'py.init_buffer_bytes': 1024**2 * 50
  }
)
tiledb.default_ctx(cfg)

tiledb.Ctx() [see Ctx.config() for configuration]

Below are the names of the arrays we will create. If you are running this on TileDB Cloud, you should use `~/` to store everything in the home directory of your dedicated EBS volume.

In [8]:
if not os.path.exists("data"):
    os.mkdir("data")
array_dense_1 = os.path.expanduser("data/array_dense_1")
array_dense_2 = os.path.expanduser("data/array_dense_2")
array_dense_3 = os.path.expanduser("data/array_dense_3")

Remove the arrays if they already exist.

In [9]:
if os.path.exists(array_dense_1):
    shutil.rmtree(array_dense_1)
if os.path.exists(array_dense_2):
    shutil.rmtree(array_dense_2)
if os.path.exists(array_dense_3):
    shutil.rmtree(array_dense_3)

## A simple 2D dense array

We will create a 2D dense array, with dimensions `d1` and `d2` and domains `[1,4]`. The array will also have a single integer attribute `a`.

In [10]:
# Create the two dimensions
d1 = tiledb.Dim(name="d1", domain=(1, 4), tile=2, dtype=np.int32)
d2 = tiledb.Dim(name="d2", domain=(1, 4), tile=2, dtype=np.int32)

# Create a domain using the two dimensions
dom1 = tiledb.Domain(d1, d2)

# Create an attribute
a = tiledb.Attr(name="a", dtype=np.int32)

# Create the array schema, setting `sparse=False` to indicate a dense array
schema1 = tiledb.ArraySchema(domain=dom1, sparse=False, attrs=[a])

# Create the array on disk (it will initially be empty)
tiledb.Array.create(array_dense_1, schema1)

Let's view the contents of the array so far (we use the `tree` package):

In [12]:
!tree $array_dense_1

data/array_dense_1
├── __commits
├── __fragment_meta
├── __fragments
├── __meta
└── __schema
    └── __1662754126687_1662754126687_48ef60b4285d4bfab2201b05d7b893da

5 directories, 1 file


The array does not contain any data yet. It only has an array schema file inside the `__schema` folder that describes the array (e.g., the number of dimensions, their names and types, etc).

To inspect the array schema, simply run:

In [13]:
# Read the array schema
schema = tiledb.ArraySchema.load(array_dense_1)
schema

ArraySchema(
  domain=Domain(*[
    Dim(name='d1', domain=(1, 4), tile=2, dtype='int32'),
    Dim(name='d2', domain=(1, 4), tile=2, dtype='int32'),
  ]),
  attrs=[
    Attr(name='a', dtype='int32', var=False, nullable=False),
  ],
  cell_order='row-major',
  tile_order='row-major',
  capacity=10000,
  sparse=False,
)

Now let's write some data to the array, using a 2D numpy array:

In [14]:
# Prepare some data in a numpy array
data = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]], dtype=np.int32)
data

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]], dtype=int32)

In [15]:
# Open the array in write mode and write to the whole array domain
with tiledb.open(array_dense_1, 'w') as A:
    A[:] = data

Let's inspect the array again:

In [16]:
!tree $array_dense_1

data/array_dense_1
├── __commits
│   └── __1662754250166_1662754250166_d801aa0d50be4dbcb9f85488de467590_15.wrt
├── __fragment_meta
├── __fragments
│   └── __1662754250166_1662754250166_d801aa0d50be4dbcb9f85488de467590_15
│       ├── __fragment_metadata.tdb
│       └── a0.tdb
├── __meta
└── __schema
    └── __1662754126687_1662754126687_48ef60b4285d4bfab2201b05d7b893da

6 directories, 4 files


Now there is a fragment directory in the `fragments` folder and a commit file with the same name and suffix `.wrt` in the `commits` folder. 

Let's read the array:

In [17]:
# Open the array in read mode and read the whole array
A = tiledb.open(array_dense_1, 'r')

In [19]:
print(A[:])        # dictionary of 2D numpy arrays

OrderedDict([('a', array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]], dtype=int32))])


In [20]:
print(A[:]['a'])   # numpy array

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]


Note that the result of `A[:]` is a dictionary containing a 2D numpy array per attribute.

We can also efficiently slice a portion of the array (very useful when the arrays are too big to fit in main memory):

In [38]:
# ranges start at 1 (unlike Python), the end is exclusive (like Python)
print(A[1:3, 1:2]["a"])
print(A[1:3, :]["a"])
print(A[:, 1:2]["a"])

[[1]
 [5]]
[[1 2 3 4]
 [5 6 7 8]]
[[ 1]
 [ 5]
 [ 9]
 [13]]


We can even slice a **multi-rage subarray** (note that `multi_index` uses *closed ranges*).

In [39]:
print(A.multi_index[[slice(1,2), 4], slice(1,3)]["a"])

[[ 1  2  3]
 [ 5  6  7]
 [13 14 15]]


Remember to close the array.

In [40]:
A.close()

## Array metadata

You can also attach any **key-value** metadata to an array:

In [41]:
# Open the array for writing
with tiledb.open(array_dense_1, "w") as A:
    A.meta["author"] = "Stavros"
    A.meta["volume"] = 2.1
    # multiple values of the same type may be written as a tuple:
    A.meta["tuple_int"] = (1,2,3,4)

Let's inspect the array again:

In [42]:
!tree $array_dense_1

data/array_dense_1
├── __commits
│   └── __1662754250166_1662754250166_d801aa0d50be4dbcb9f85488de467590_15.wrt
├── __fragment_meta
├── __fragments
│   └── __1662754250166_1662754250166_d801aa0d50be4dbcb9f85488de467590_15
│       ├── __fragment_metadata.tdb
│       └── a0.tdb
├── __meta
│   └── __1662754951764_1662754951764_b5692b3cc0b1479eaca12a8c7824dbeb
└── __schema
    └── __1662754126687_1662754126687_48ef60b4285d4bfab2201b05d7b893da

6 directories, 5 files


Now observe that there is a new file inside the `__meta` folder, which contains the key-value pairs we have written.

Let's read those values back from the array:

In [43]:
# Open the array for reading
with tiledb.open(array_dense_1, "r") as A:
    # print values from specific keys
    print(A.meta["author"])
    print(A.meta["volume"])
    print(A.meta["tuple_int"])
    
    # print all keys:
    print(A.meta.keys())

Stavros
2.1
(1, 2, 3, 4)
['author', 'tuple_int', 'volume']


## Storing multiple attributes

TileDB allows you to store more than one values in each cell, potentially of different types. This means that the array can have more than one attributes. TileDB is a **"columnar"** format, in that it stores the values of each attribute in a separate file, allowing for better compression and faster attribute subselection.

Let's create the same 2D array, but now with an extra `char` attribute.

In [44]:
# Create the two dimensions
d1 = tiledb.Dim(name="d1", domain=(1, 4), tile=2, dtype=np.int32)
d2 = tiledb.Dim(name="d2", domain=(1, 4), tile=2, dtype=np.int32)

# Create a domain using the two dimensions
dom2 = tiledb.Domain(d1, d2)

# Create two attributes
a1 = tiledb.Attr(name="a1", dtype=np.int32)
a2 = tiledb.Attr(name="a2", dtype="S1")

# Create the array schema, setting `sparse=False` to indicate a dense array
schema2 = tiledb.ArraySchema(domain=dom2, sparse=False, attrs=[a1, a2])

# Create the array on disk (it will initially be empty)
tiledb.Array.create(array_dense_2, schema2)

Let's inspect the array schema to see the two attributes listed:

In [45]:
# Read the array schema
schema = tiledb.ArraySchema.load(array_dense_2)
schema

ArraySchema(
  domain=Domain(*[
    Dim(name='d1', domain=(1, 4), tile=2, dtype='int32'),
    Dim(name='d2', domain=(1, 4), tile=2, dtype='int32'),
  ]),
  attrs=[
    Attr(name='a1', dtype='int32', var=False, nullable=False),
    Attr(name='a2', dtype='|S1', var=False, nullable=False),
  ],
  cell_order='row-major',
  tile_order='row-major',
  capacity=10000,
  sparse=False,
)

Now let's prepare and write some data to the array:

In [46]:
# Prepare some data for the first attribute
a1_data = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]], dtype=np.int32)
a1_data

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]], dtype=int32)

In [47]:
# Prepare some data for the second attribute
a2_data = np.array([
    ['a', 'b', 'c', 'd'],
    ['e', 'f', 'g', 'h'],
    ['i', 'j', 'k', 'l'],
    ['m', 'n', 'o', 'p']], dtype="S1")
a2_data

array([[b'a', b'b', b'c', b'd'],
       [b'e', b'f', b'g', b'h'],
       [b'i', b'j', b'k', b'l'],
       [b'm', b'n', b'o', b'p']], dtype='|S1')

In [48]:
# Write an ordered dictionary, passing one numpy array per attribute
with tiledb.open(array_dense_2, 'w') as A:
    A[:] = {'a1': a1_data, 'a2': a2_data}

Let's read all the data back:

In [49]:
A = tiledb.open(array_dense_2, 'r')

In [50]:
print(A[:]["a2"])

[[b'a' b'b' b'c' b'd']
 [b'e' b'f' b'g' b'h']
 [b'i' b'j' b'k' b'l']
 [b'm' b'n' b'o' b'p']]


The above always reads **all** attributes for the given slice. If you wish to read a subset of attributes, there is a faster way that allows fetching only the desired data:

In [51]:
print(A.query(attrs=['a1'])[:])
print(A.query(attrs=['a2'])[:])

OrderedDict([('a1', array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]], dtype=int32))])
OrderedDict([('a2', array([[b'a', b'b', b'c', b'd'],
       [b'e', b'f', b'g', b'h'],
       [b'i', b'j', b'k', b'l'],
       [b'm', b'n', b'o', b'p']], dtype='|S1'))])


## Variable-length attributes

TileDB supports also variable-length attributes, such as strings. Here is the same 2D array we used above, but now having a single string attribute.

In [52]:
# Create the two dimensions
d1 = tiledb.Dim(name="d1", domain=(1, 4), tile=2, dtype=np.int32)
d2 = tiledb.Dim(name="d2", domain=(1, 4), tile=2, dtype=np.int32)

# Create a domain using the two dimensions
dom3 = tiledb.Domain(d1, d2)

# Create a string attribute
a = tiledb.Attr(name="a", dtype="S0")

# Create the array schema, setting `sparse=False` to indicate a dense array
schema3 = tiledb.ArraySchema(domain=dom3, sparse=False, attrs=[a])

# Create the array on disk (it will initially be empty)
tiledb.Array.create(array_dense_3, schema3)

Let's inspect the array schema:

In [53]:
# Read the array schema
schema = tiledb.ArraySchema.load(array_dense_3)
schema

ArraySchema(
  domain=Domain(*[
    Dim(name='d1', domain=(1, 4), tile=2, dtype='int32'),
    Dim(name='d2', domain=(1, 4), tile=2, dtype='int32'),
  ]),
  attrs=[
    Attr(name='a', dtype='|S0', var=True, nullable=False),
  ],
  cell_order='row-major',
  tile_order='row-major',
  capacity=10000,
  sparse=False,
)

Here is how we write to arrays with string attributes:

In [54]:
# Prepare some data
a_data = np.array([
    ["a", "bb", "ccc", "dddd"],
    ["e", "ff", "ggg", "hhhh"],
    ["i", "jj", "kkk", "llll"],
    ["m", "nn", "ooo", "pppp"]], dtype=object)
a_data

array([['a', 'bb', 'ccc', 'dddd'],
       ['e', 'ff', 'ggg', 'hhhh'],
       ['i', 'jj', 'kkk', 'llll'],
       ['m', 'nn', 'ooo', 'pppp']], dtype=object)

In [55]:
# Write to the array
with tiledb.open(array_dense_3, 'w') as A:
    A[:] = a_data

Let's inspect the array:

In [56]:
!tree $array_dense_3

data/array_dense_3
├── __commits
│   └── __1662758492536_1662758492536_1580272592b54f438b544f4c02e7980f_15.wrt
├── __fragment_meta
├── __fragments
│   └── __1662758492536_1662758492536_1580272592b54f438b544f4c02e7980f_15
│       ├── __fragment_metadata.tdb
│       ├── a0.tdb
│       └── a0_var.tdb
├── __meta
└── __schema
    └── __1662758474274_1662758474274_a82446a5074447a691aafb2592fdaa90

6 directories, 5 files


Observe that TileDB stores **two files** for a variable-length attribute inside the written fragment folder: `a0_var.tdb` that stores the actual variable-length cell values, and `a0.tdb` that stores the starting offset of each of the cell values for fast identification retrieval.

Now let's perform a couple of read queries:

In [57]:
with tiledb.open(array_dense_3, 'r') as A:
    print(A[:]["a"]) # whole array
    print(A[1:3, 1:2]["a"]) # slice

[[b'a' b'bb' b'ccc' b'dddd']
 [b'e' b'ff' b'ggg' b'hhhh']
 [b'i' b'jj' b'kkk' b'llll']
 [b'm' b'nn' b'ooo' b'pppp']]
[[b'a']
 [b'e']]


## Arrays as dataframes

Arrays are essentially dataframes where dimensions are special (indexed) columns that allow very fast slicing. Revisiting the very first array we created, we can also slice it via the `df` object that returns the results in a `pandas` dataframe:

In [58]:
A = tiledb.open(array_dense_1, 'r')

In [59]:
A.df[:]        # whole array

,d1,d2,a
0,1,1,1
1,1,2,2
2,1,3,3
3,1,4,4
4,2,1,5
5,2,2,6
6,2,3,7
7,2,4,8
8,3,1,9
9,3,2,10


In [60]:
A.df[1:3, 1:2] # slice

,d1,d2,a
0,1,1,1
1,1,2,2
2,2,1,5
3,2,2,6
4,3,1,9
5,3,2,10


We can also subselect on the dimensions and attributes via the `query` object:

In [61]:
A.query(attrs=[], dims=["d1"]).df[:]        # whole array

,d1
0,1
1,1
2,1
3,1
4,2
5,2
6,2
7,2
8,3
9,3


In [62]:
A.query(attrs=[], dims=["d1"]).df[1:3, 1:2] # slice

,d1
0,1
1,1
2,2
3,2
4,3
5,3


Note that the ranges inside the `df` object are **inclusive**.

In [63]:
pd.read_sql(sql=f"select * from `{array_dense_1}`", con=db)

/home/jdblischak/miniconda3/envs/tiledb/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,d1,d2,a
0,1,1,1
1,1,2,2
2,1,3,3
3,1,4,4
4,2,1,5
5,2,2,6
6,2,3,7
7,2,4,8
8,3,1,9
9,3,2,10


In [64]:
pd.read_sql(sql=f"select a from `{array_dense_1}` where d2 >= 2", con=db)

/home/jdblischak/miniconda3/envs/tiledb/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,a
0,2
1,3
2,4
3,6
4,7
5,8
6,10
7,11
8,12
9,14


Remember to close the array.

In [65]:
A.close()

In [41]:
# Optionally delete the created arrays
# if os.path.exists(array_dense_1):
#    shutil.rmtree(array_dense_1)
# if os.path.exists(array_dense_2):
#    shutil.rmtree(array_dense_2)
# if os.path.exists(array_dense_3):
#    shutil.rmtree(array_dense_3)